In [ ]:
"""
Question: if you take any vector from the embedding metric,
is it true that its dot-product will be maximal with itself, and with other vectors - less?
In other words, if you encode a token into a vector, then is it decoded back using argmax.
Obviously, if all vectors are normalized, then yes - the dot product with itself gives 1, and with others - less than 1.
But for real embedding matrices this is not the case.
In Llama, about 500 out of 128,000 vectors are not decoded back.
And basically these are either tokens not from English, or tokens from Python like \t\t\t\t\t\t\t\t\r\n, or reserved_special_token.
If you do the same with the head of the model, there are more such errors - ~800, some long tokens from the English language are added there.
It's also interesting that after normalizing everything with a vector,
everything fell into place for the input embedding matrix, but head still gave 200 discrepancies,
apparently due to a numerical error, if I didn't impose it anywhere.
"""

In [1]:
import torch
import seaborn as sns

from emb_vectors_functions import find_self_embeds, get_model_and_embed
from emb_dist_functions import get_tokens_from_vectors, plot_dist

ImportError: cannot import name 'get_model_and_embed' from 'emb_vectors_functions' (/home/galimzyanov/llm-experiments/short_experiments/emb_vectors_functions.py)

In [ ]:
def get_norms(tensor):

    norms = torch.norm(tensor, p=2, dim=1)
    print(norms.shape)
    average_norm = norms.mean().item()
    std_norm = norms.std()
    print(f"Average Norm: {average_norm}")
    print(f"Standard Deviation of Norms: {std_norm}")
    print(f"std/mead: {std_norm/average_norm}")

In [ ]:
# Loading embeddings:
model_name = "meta-llama/Meta-Llama-3.1-8B"  # "meta-llama/Meta-Llama-3.1-8B" , "gpt2", "meta-llama/Llama-2-7b-hf"
model, embedding, head, model_norm, mean_norm, tokenizer = get_model_and_embed(
    model_name
)

embedding_norm = (embedding.T / torch.norm(embedding, p=2, dim=1)).T
head_norm = (head.T / torch.norm(head, p=2, dim=1)).T
embedding_rms_norm = model_norm(embedding)
head_rms_norm = model_norm(head)

In [ ]:
tokens_dist = get_tokens_from_vectors(
    head, batch_size=1000, num_batches=1000, do_rms=False, model_norm=model_norm
)

In [ ]:
token_counts = plot_dist(tokens_dist)

In [ ]:
decoded_counts = {
    tokenizer.decode(token_id): count for token_id, count in token_counts.items()
}
decoded_counts = dict(
    sorted(decoded_counts.items(), key=lambda item: item[1], reverse=True)
)

In [ ]:
tok_freqs = list(decoded_counts.values())

In [ ]:
tok_freqs_filt = [freq for freq in tok_freqs if 0 <= freq <= 100]
sns.histplot(tok_freqs_filt, bins=300, kde=False, log=True)

In [ ]:
failed_emb, failed_res_emb, failed_pairs = find_self_embeds(
    head, tokenizer, head_rms_norm
)

In [ ]:
failed_emb = find_self_embeds(embedding, embedding)
print(len(failed_emb))
failed_emb_norm = find_self_embeds(embedding_norm, embedding_norm)
print(len(failed_emb))

failed_head = find_self_embeds(head, head)

failed_emb_toks = [tokenizer.decode(idx) for idx in failed_emb]
failed_head_toks = [tokenizer.decode(idx) for idx in failed_head]

joined_set = set(failed_emb_toks) | set(failed_head_toks)

In [ ]:
failed_emb = find_self_embeds(embedding, embedding_norm)